In [1]:
import numpy as np
import torch
import json
import faiss
from transformers import AutoTokenizer, AutoModel

# Bước 1: Lấy embedding cho câu hỏi
def get_query_embedding(query, model, tokenizer, device):
    encoded = tokenizer(query, return_tensors="pt", padding=True, truncation=True, max_length=512)
    encoded = {k: v.to(device) for k, v in encoded.items()}
    
    with torch.no_grad():
        outputs = model(**encoded)
        emb = outputs.last_hidden_state[:, 0, :]  # CLS token
        emb = torch.nn.functional.normalize(emb, p=2, dim=1)  # L2 normalize
    return emb.cpu().numpy()

# Bước 2: Truy vấn FAISS Index
def search_faiss(index, query_embedding, k=5):
    D, I = index.search(query_embedding, k)  # D là độ tương đồng, I là chỉ số (indices) các vector gần nhất
    return D, I

c:\Users\thain\anaconda3\envs\ragbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Bước 3: In kết quả
def print_results(indices, metadata, D):
    for i, idx in enumerate(indices[0]):
        print(f"Rank {i+1}:")
        print(f"Metadata: {metadata[idx]}")
        print(f"Cosine similarity: {D[0][i]:.4f}")
        print("-" * 40)

# ----
# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model và tokenizer
model_name = "intfloat/multilingual-e5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).eval().to(device)

# Load FAISS index và metadata
index = faiss.read_index("wiki_chunks.index")

with open("wiki_metadata.json", "r", encoding="utf-8") as f:
    metadata = json.load(f)

c:\Users\thain\anaconda3\envs\ragbot\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# Bước 4: Test với câu hỏi
def test_query(query, model, tokenizer, index, metadata, device, k=5):
    print(f"\n❓ Câu hỏi: {query}")
    query_embedding = get_query_embedding(query, model, tokenizer, device)
    D, I = search_faiss(index, query_embedding, k=k)
    print_results(I, metadata, D)

# Danh sách câu hỏi để test
test_questions = [
    "Thủ đô của Việt Nam nằm ở đâu?",
    "Sông nào dài nhất Việt Nam?",
    "Đỉnh núi cao nhất Việt Nam là gì?",
    "Biển Đông giáp những tỉnh nào của Việt Nam?",
    "Vịnh Hạ Long thuộc tỉnh nào?",

    "Bác Hồ đọc Tuyên ngôn Độc lập vào ngày nào?",
    "Trận Điện Biên Phủ diễn ra vào năm nào?",
    "Ai là vị vua đầu tiên của nước Việt Nam?",
    "Vua Quang Trung nổi tiếng với trận đánh nào?",
    "Khởi nghĩa Hai Bà Trưng diễn ra năm nào?"
]

# Chạy tất cả câu hỏi
for question in test_questions:
    test_query(question, model, tokenizer, index, metadata, device, k=5)


❓ Câu hỏi: Thủ đô của Việt Nam nằm ở đâu?
Rank 1:
Metadata: {'title': 'Niên hiệu Việt Nam', 'category': 'Lịch sử Việt Nam', 'timestamp': '2025-04-24 17:54:10', 'chunk_id': 0, 'total_chunks': 5, 'url': 'https://vi.wikipedia.org/wiki/Niên_hiệu_Việt_Nam'}
Cosine similarity: 0.8819
----------------------------------------
Rank 2:
Metadata: {'title': 'Thành phố (Việt Nam)', 'category': 'Thành phố của Việt Nam', 'timestamp': '2025-04-24 18:41:13', 'chunk_id': 12, 'total_chunks': 13, 'url': 'https://vi.wikipedia.org/wiki/Thành_phố_(Việt_Nam)'}
Cosine similarity: 0.8730
----------------------------------------
Rank 3:
Metadata: {'title': 'Danh sách đồng bằng Việt Nam', 'category': 'Danh sách địa lý Việt Nam', 'timestamp': '2025-04-24 18:08:20', 'chunk_id': 12, 'total_chunks': 13, 'url': 'https://vi.wikipedia.org/wiki/Danh_sách_đồng_bằng_Việt_Nam'}
Cosine similarity: 0.8730
----------------------------------------
Rank 4:
Metadata: {'title': 'Bắc Bộ', 'category': 'Miền Việt Nam', 'timestamp': 